In [5]:
with open("input.txt", "r", encoding="utf-8") as f:
  text = f.read()

print("length of dataset in characters:", len(text))
print(text[:400])

length of dataset in characters: 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it 


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [10]:
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [11]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:400])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [12]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [16]:
x = train_data[:block_size]
y = train_data[1 : block_size + 1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"{context.tolist()} --> {target}")

[18] --> 47
[18, 47] --> 56
[18, 47, 56] --> 57
[18, 47, 56, 57] --> 58
[18, 47, 56, 57, 58] --> 1
[18, 47, 56, 57, 58, 1] --> 15
[18, 47, 56, 57, 58, 1, 15] --> 47
[18, 47, 56, 57, 58, 1, 15, 47] --> 58


In [21]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  # generate a small batch of data of inputs x and targets y
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y


xb, yb = get_batch("train")
xb.shape, xb, yb.shape, yb

(torch.Size([4, 8]),
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 torch.Size([4, 8]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]))

In [48]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    # idx and targets are (B, T) of integers
    logits = self.token_embedding_table(idx) # (B, T, C)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices
    for _ in range(max_new_tokens):
      # get predictions
      logits, _ = self(idx) # (B, T, C)
      # focus only on the last time step
      logits = logits[:, -1, :] # (B, C)
      # get probabilities
      probs = F.softmax(logits, dim=-1)
      # sample
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append to the context
      idx = torch.cat((idx, idx_next), dim=-1)
    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(loss)
print(logits.shape)
print(logits)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

tensor(4.7805, grad_fn=<NllLossBackward0>)
torch.Size([256, 65])
tensor([[-1.3441, -0.2827, -0.6887,  ...,  1.4385, -1.3166,  1.2690],
        [ 0.1808, -0.0700, -0.3596,  ...,  1.6097, -0.4032, -0.8345],
        [ 0.7029,  1.4840,  0.1137,  ..., -0.5048,  0.8791,  0.4086],
        ...,
        [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
        [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [ 0.4897,  0.0655,  1.0370,  ...,  0.4397, -0.7343, -0.1916]],
       grad_fn=<ViewBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [49]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [55]:
batch_size = 32
for steps in range(10000):
  xb, yb = get_batch("train")
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())

2.4527838230133057


In [58]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))


Fourigrlomson En; by; me t?
manchure.
CERoof Ye, INCEShe h hatharvecowitofochand tee oucough s; he als'd:
Alens iostist, t med thesthoa us, m s an;
An ase

heruerdeve-n o'sk;
GAnouilerrlince hane s s arderous.

NII apif ise thanf fot Se,
Pavest omand foar y d fed:
't hour atig h 'twalillorioll mend wr pptor sbe; a,
Ithaco atlermyothos imost oensurinainclun p
OMIf msin bel ar tut ry hatharcthau fe ifuth wit, is ceeal nste has agisser n
Bores lenons tha t!
S:
's, o d the ge n fopasthens
HA puis li
